In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
from io import StringIO
import sklearn as skn
from scipy import stats
import pandas_datareader.data as pdr
import riskfolio as rp
from scipy.stats import multivariate_normal


In [2]:
def loadStooqData(ticker: str,start, frequency='d'):
    url = f'https://stooq.pl/q/d/l/?s={ticker}&i={frequency}'
    response = requests.get(url)
    data = pd.read_csv(StringIO(response.text))
    data.set_index('Data', inplace=True)
    data = data[start:]
    return data['Zamkniecie']

def loadYahooData(ticker, start, frequency = '1d'):
    ticker  = yf.Ticker(ticker)
    hist = ticker.history(start=start,interval=frequency)
    price = hist['Close']
    return price

def sharp(returns):
    return returns.mean()/returns.std()

def assets_performance(returns: pd.DataFrame):
    return returns.agg(['mean', 'std', 'median', 'skew', 'kurtosis', sharp])

def beta(X, Y):

    X_cov = X.cov()

    XY = np.append(X,np.expand_dims(Y,axis=1),axis=0)
    XY_cov = np.cov(XY)

    #XY_sub = XY_cov.iloc[-1,0:2]

    #inv = np.linalg.inv(X_cov)
    #B = inv @ XY_sub
    return  XY_cov

In [3]:
yahoo_tickers = ['ACWI', 'CORP.L']

stooq_tickers = ['XAUPLN', '^TBSP', 'MWIG40TR', 'USDPLN', 'EURPLN', 'PLOPLN3M']

#na przyszłość EAFA,EEM,ACWI



In [4]:
base_prices = pd.DataFrame({})
start = '2008-01-01'

for ticker in stooq_tickers:
    base_prices[ticker] = loadStooqData(ticker,start)

for ticker in yahoo_tickers:
    base_prices[ticker] = loadYahooData(ticker,start)

base_prices.index = pd.DatetimeIndex(base_prices.index)


In [5]:
#get monthly data
monthly_base_prices = base_prices.resample('m').last()

In [6]:
monthly_base_returns = np.log(monthly_base_prices/monthly_base_prices.shift(1))
assets_performance(monthly_base_returns)

,XAUPLN,^TBSP,MWIG40TR,USDPLN,EURPLN,PLOPLN3M,ACWI,CORP.L
mean,0.006611,0.003242,0.007140,0.002797,0.001137,0.000904,0.005075,-0.001271
std,0.053810,0.012801,0.050648,0.040147,0.021726,0.149127,0.050719,0.019787
median,0.003995,0.003729,0.010507,0.000648,-0.000957,0.000000,0.011983,0.001964
skew,0.603117,0.207682,-0.639711,0.790070,0.668567,2.912139,-0.750764,-0.688823
kurtosis,1.896176,5.206100,2.714113,2.003371,2.590691,38.493637,1.474475,2.513197
sharp,0.122861,0.253235,0.140963,0.069661,0.052354,0.006059,0.100054,-0.064240


In [7]:
monthly_base_returns.corr()

,XAUPLN,^TBSP,MWIG40TR,USDPLN,EURPLN,PLOPLN3M,ACWI,CORP.L
XAUPLN,1.000000,0.086884,-0.254582,0.486515,0.525404,-0.020591,-0.381192,0.036114
^TBSP,0.086884,1.000000,0.110798,-0.202122,-0.102806,-0.383637,0.232823,0.545680
MWIG40TR,-0.254582,0.110798,1.000000,-0.520156,-0.557332,-0.052983,0.747081,0.554543
USDPLN,0.486515,-0.202122,-0.520156,1.000000,0.751418,0.018977,-0.714470,-0.595445
EURPLN,0.525404,-0.102806,-0.557332,0.751418,1.000000,-0.028350,-0.567597,-0.421699
PLOPLN3M,-0.020591,-0.383637,-0.052983,0.018977,-0.028350,1.000000,-0.021751,-0.117238
ACWI,-0.381192,0.232823,0.747081,-0.714470,-0.567597,-0.021751,1.000000,0.691530
CORP.L,0.036114,0.545680,0.554543,-0.595445,-0.421699,-0.117238,0.691530,1.000000


In [8]:
#calculate pln returns
pln_prices = monthly_base_prices
pln_prices['ACWI'] = monthly_base_prices['ACWI'] * monthly_base_prices['USDPLN']
pln_prices['CORP.L'] = monthly_base_prices['CORP.L'] * monthly_base_prices['USDPLN']
pln_returns = np.log(pln_prices/pln_prices.shift(1))
pln_returns['PLOPLN3M'] = pln_prices['PLOPLN3M'] / 100 / 12
assets_performance(pln_returns)

,XAUPLN,^TBSP,MWIG40TR,USDPLN,EURPLN,PLOPLN3M,ACWI,CORP.L
mean,0.006611,0.003242,0.007140,0.002797,0.001137,0.002665,0.008377,0.000638
std,0.053810,0.012801,0.050648,0.040147,0.021726,0.001677,0.035692,0.023704
median,0.003995,0.003729,0.010507,0.000648,-0.000957,0.002242,0.008943,0.000866
skew,0.603117,0.207682,-0.639711,0.790070,0.668567,0.476426,-0.265203,0.240951
kurtosis,1.896176,5.206100,2.714113,2.003371,2.590691,-0.820268,0.635426,-0.362800
sharp,0.122861,0.253235,0.140963,0.069661,0.052354,1.589324,0.234697,0.026933


In [9]:
pln_returns.corr()

,XAUPLN,^TBSP,MWIG40TR,USDPLN,EURPLN,PLOPLN3M,ACWI,CORP.L
XAUPLN,1.000000,0.086884,-0.254582,0.486515,0.525404,0.019802,0.000934,0.487419
^TBSP,0.086884,1.000000,0.110798,-0.202122,-0.102806,0.130142,0.094272,0.162872
MWIG40TR,-0.254582,0.110798,1.000000,-0.520156,-0.557332,-0.054362,0.430288,-0.105879
USDPLN,0.486515,-0.202122,-0.520156,1.000000,0.751418,0.039240,0.107124,0.741781
EURPLN,0.525404,-0.102806,-0.557332,0.751418,1.000000,0.015293,0.036666,0.552887
PLOPLN3M,0.019802,0.130142,-0.054362,0.039240,0.015293,1.000000,-0.155221,-0.151796
ACWI,0.000934,0.094272,0.430288,0.107124,0.036666,-0.155221,1.000000,0.484333
CORP.L,0.487419,0.162872,-0.105879,0.741781,0.552887,-0.151796,0.484333,1.000000


In [10]:
pln_returns.mean() * 12

XAUPLN      0.079334
^TBSP       0.038900
MWIG40TR    0.085674
USDPLN      0.033560
EURPLN      0.013649
PLOPLN3M    0.031977
ACWI        0.100521
CORP.L      0.007661
dtype: float64

In [11]:
expected_returns = pd.Series({
'ACWI': 0.085/12,
'^TBSP': 0.035/12,
'XAUPLN': 0.054/12,
'MWIG40TR': 0.080/12,
'CORP.L': 0.037/12,
'PLOPLN3M': 0.03/12
})


In [42]:
assets = ['ACWI', '^TBSP', 'XAUPLN', 'MWIG40TR', 'CORP.L', 'PLOPLN3M']
#
cov_matrix = pln_returns[assets].cov()

* load data
* describe data in local currency
* transform data do pln
* describe data in pln

In [43]:
portfolio = rp.Portfolio(pd.DataFrame(pln_returns[assets]))
#pln_returns[assets]

portfolio.mu = expected_returns
portfolio.cov = cov_matrix

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'MinRisk' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = False # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = portfolio.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

,ACWI,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M
weights,0.004058,1.149482e-07,2.768932e-08,0.002216,0.012566,0.98116


In [44]:
w = portfolio.optimization(model=model, rm=rm, obj='MaxRet', rf=rf, l=l, hist=hist)

display(np.round(w.T,2))

,ACWI,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M
weights,1.0,0.0,0.0,0.0,0.0,0.0


In [60]:
def frontier_performance(frontier,mu,cov,rf = 0.0):
    result = frontier.copy()
    result['Returns'] = np.round(frontier.to_numpy() @ mu *12,4)
    result['Std. deviations'] = frontier.apply(lambda x: np.round(np.sqrt(x.to_numpy().T @ cov @ x.to_numpy())* np.sqrt(12),4),axis=1)
    result['SR'] = np.round((result['Returns']-rf) / result['Std. deviations'],2)
    return result


In [46]:
def shrinkedCovariance(returns: pd.DataFrame, w: int):
    std = np.diag(returns.std())
    corr = returns.corr('pearson')
    shrinked_corr = (1-w)*corr + np.ones_like(corr)*w
    result = std @ shrinked_corr @ std
    return result

shrin_param = 0.75

In [47]:
points = 10
frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_10 = np.round(frontier.T,2)

In [63]:
frontier_performance(frontier_10, expected_returns,cov_matrix,0.0298)

,ACWI,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.00,0.00,0.00,0.00,0.01,0.98,0.0298,0.0056,0.00
1,0.12,0.03,0.03,0.04,0.00,0.79,0.0398,0.0193,0.52
2,0.20,0.05,0.06,0.06,0.00,0.63,0.0457,0.0318,0.50
3,0.28,0.08,0.08,0.09,0.00,0.47,0.0522,0.0451,0.50
4,0.36,0.10,0.10,0.11,0.00,0.32,0.0579,0.0573,0.49
5,0.44,0.12,0.13,0.14,0.00,0.17,0.0649,0.0711,0.49
6,0.52,0.15,0.15,0.17,0.00,0.02,0.0718,0.0845,0.50
7,0.61,0.02,0.17,0.20,0.00,0.00,0.0777,0.0977,0.49
8,0.78,0.00,0.06,0.16,0.00,0.00,0.0823,0.1105,0.48
9,1.00,0.00,0.00,0.00,0.00,0.00,0.0850,0.1236,0.45


In [49]:
mu = expected_returns
shrinked_cov = shrinkedCovariance(pln_returns[assets],shrin_param)

portfolio.mu = mu
portfolio.cov = shrinked_cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_10_sh = np.round(frontier.T,2)

In [64]:
frontier_performance(frontier_10_sh,mu,shrinked_cov,0.0298)


,ACWI,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.00,0.0,0.0,0.0,0.0,1.00,0.0300,0.0017,0.12
1,0.12,0.0,0.0,0.0,0.0,0.88,0.0366,0.0129,0.53
2,0.23,0.0,0.0,0.0,0.0,0.77,0.0426,0.0235,0.54
3,0.34,0.0,0.0,0.0,0.0,0.66,0.0487,0.0342,0.55
4,0.45,0.0,0.0,0.0,0.0,0.55,0.0548,0.0448,0.56
5,0.56,0.0,0.0,0.0,0.0,0.44,0.0608,0.0554,0.56
6,0.67,0.0,0.0,0.0,0.0,0.33,0.0669,0.0660,0.56
7,0.78,0.0,0.0,0.0,0.0,0.22,0.0729,0.0767,0.56
8,0.89,0.0,0.0,0.0,0.0,0.11,0.0790,0.0873,0.56
9,1.00,0.0,0.0,0.0,0.0,0.00,0.0850,0.0979,0.56


In [51]:
mu = expected_returns
cov = pln_returns[0:60][assets].cov()

portfolio.mu = mu
portfolio.cov = cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_1_5 = np.round(frontier.T,2)

In [52]:
frontier_performance(frontier_1_5,mu,cov)

,ACWI,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.01,0.00,0.00,0.00,0.00,0.99,0.0306,0.0025,12.24
1,0.09,0.08,0.03,0.06,0.24,0.50,0.0407,0.0170,2.39
2,0.17,0.17,0.05,0.11,0.47,0.04,0.0505,0.0319,1.58
3,0.25,0.00,0.08,0.19,0.48,0.00,0.0585,0.0461,1.27
4,0.33,0.00,0.11,0.26,0.30,0.00,0.0659,0.0604,1.09
5,0.41,0.00,0.14,0.32,0.13,0.00,0.0728,0.0752,0.97
6,0.50,0.00,0.13,0.36,0.00,0.00,0.0783,0.0887,0.88
7,0.65,0.00,0.03,0.32,0.00,0.00,0.0825,0.1036,0.80
8,0.86,0.00,0.00,0.14,0.00,0.00,0.0843,0.1187,0.71
9,1.00,0.00,0.00,0.00,0.00,0.00,0.0850,0.1329,0.64


In [53]:
mu = expected_returns
shrinked_cov = shrinkedCovariance(pln_returns[0:60][assets],shrin_param)

portfolio.mu = mu
portfolio.cov = shrinked_cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_1_5_sh = np.round(frontier.T,2)

In [54]:
frontier_performance(frontier_1_5_sh,mu,shrinked_cov)

,ACWI,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.00,0.0,0.0,0.0,0.00,1.00,0.0300,0.0025,12.00
1,0.11,0.0,0.0,0.0,0.00,0.89,0.0360,0.0162,2.22
2,0.23,0.0,0.0,0.0,0.00,0.77,0.0426,0.0319,1.34
3,0.34,0.0,0.0,0.0,0.00,0.66,0.0487,0.0463,1.05
4,0.45,0.0,0.0,0.0,0.01,0.54,0.0548,0.0609,0.90
5,0.55,0.0,0.0,0.0,0.03,0.42,0.0605,0.0744,0.81
6,0.66,0.0,0.0,0.0,0.04,0.29,0.0663,0.0890,0.74
7,0.77,0.0,0.0,0.0,0.06,0.17,0.0728,0.1037,0.70
8,0.88,0.0,0.0,0.0,0.07,0.05,0.0789,0.1183,0.67
9,1.00,0.0,0.0,0.0,0.00,0.00,0.0850,0.1329,0.64


In [55]:
mu = expected_returns
cov = pln_returns[60:120][assets].cov()

portfolio.mu = mu
portfolio.cov = cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_5_10 = np.round(frontier.T,2)

In [56]:
frontier_performance(frontier_5_10,mu,cov)

,ACWI,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.00,0.00,0.00,0.00,0.0,1.00,0.0300,0.0017,17.65
1,0.09,0.08,0.01,0.05,0.0,0.78,0.0384,0.0126,3.05
2,0.17,0.14,0.02,0.08,0.0,0.58,0.0442,0.0223,1.98
3,0.26,0.21,0.03,0.12,0.0,0.38,0.0521,0.0338,1.54
4,0.34,0.28,0.04,0.16,0.0,0.18,0.0591,0.0444,1.33
5,0.42,0.33,0.05,0.20,0.0,0.00,0.0660,0.0548,1.20
6,0.52,0.18,0.06,0.24,0.0,0.00,0.0729,0.0659,1.11
7,0.61,0.04,0.07,0.28,0.0,0.00,0.0794,0.0767,1.04
8,0.81,0.00,0.00,0.19,0.0,0.00,0.0841,0.0872,0.96
9,1.00,0.00,0.00,0.00,0.0,0.00,0.0850,0.0979,0.87


In [57]:
shrinked_cov = shrinkedCovariance(pln_returns[60:120][assets],shrin_param)

portfolio.mu = mu
portfolio.cov = shrinked_cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_5_10_sh = np.round(frontier.T,2)

In [58]:
frontier_performance(frontier_5_10_sh,mu,shrinked_cov)

,ACWI,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.00,0.0,0.0,0.0,0.0,1.00,0.0300,0.0017,17.65
1,0.11,0.0,0.0,0.0,0.0,0.89,0.0360,0.0120,3.00
2,0.22,0.0,0.0,0.0,0.0,0.78,0.0421,0.0226,1.86
3,0.34,0.0,0.0,0.0,0.0,0.66,0.0487,0.0342,1.42
4,0.45,0.0,0.0,0.0,0.0,0.55,0.0548,0.0448,1.22
5,0.56,0.0,0.0,0.0,0.0,0.44,0.0608,0.0554,1.10
6,0.67,0.0,0.0,0.0,0.0,0.33,0.0669,0.0660,1.01
7,0.78,0.0,0.0,0.0,0.0,0.22,0.0729,0.0767,0.95
8,0.89,0.0,0.0,0.0,0.0,0.11,0.0790,0.0873,0.90
9,1.00,0.0,0.0,0.0,0.0,0.00,0.0850,0.0979,0.87


In [59]:
acwi_tbsc = pln_returns[['ISAC.L','^TBSP']]
acwi_tbsc.describe()
np.sqrt(acwi_tbsc.std() ** 2 * 12) 

#acwi_tbsc.corr()

KeyError: "['ISAC.L'] not in index"

In [ ]:
means = expected_returns[['ISAC.L','^TBSP']]
cov = acwi_tbsc.cov()
number_of_scenarios = 1001
T = 120
sim_returns = []

for s in range (number_of_scenarios):
    sim_returns.append(multivariate_normal.rvs(means,cov, T))

sim_returns = np.array(sim_returns)

In [ ]:
sim_means = sim_returns.mean(axis=1)
sim_means.mean(0) *12

sim_std = sim_returns.std(axis=1)

sim_std.mean(0) * np.sqrt(12)

array([0.11477853, 0.04748919])

In [ ]:
e = np.arange(0,1.1,0.1)
b = 1 - e
portfolios = np.vstack((e,b)).T


In [ ]:
scenarios = sim_returns + 1
scenarios = np.cumprod(scenarios,axis=1)
r = 10
T = np.arange(11,120,12)

glide_path = []

for t in T:
    qv = []
    for p in portfolios:
        lw = scenarios[:,t] @ p
        qv.append(np.percentile(lw, r))
    glide_path.append(np.argmax(qv))

print(glide_path)
np.take(portfolios,glide_path,0)

[2, 3, 3, 3, 4, 4, 5, 4, 5, 9]


array([[0.2, 0.8],
       [0.3, 0.7],
       [0.3, 0.7],
       [0.3, 0.7],
       [0.4, 0.6],
       [0.4, 0.6],
       [0.5, 0.5],
       [0.4, 0.6],
       [0.5, 0.5],
       [0.9, 0.1]])

In [ ]:
class Simulation():
    def __init__(self, initial_value, returns, strategy) -> None:
        self.capital = initial_value
        self.returns = returns
        self.strategy = strategy 
    
   
    def step(self, current_value,weights,returns):
        current_assets_value = np.expand_dims(current_value,axis=0).T * weights
        next_assets_value =  np.round(current_assets_value * (1+returns),2)
        return next_assets_value.sum(1)
    
        
    def run(self, T):
        for t in range(T):
            self.capital = step(self.capital,self.strategy[t],self.returns[:,t])

    


In [ ]:

capital = np.full(10001,100)

T = np.arange(11,216,12)
risk = 10

glide_paths = {}

for t in T:
    perc = []
    for portfolio in range(len(portfolios)):
        strategy = np.tile(portfolios[portfolio],(216,1))
        simulator = Simulation(capital,sim_returns,strategy)
        simulator.run(t)
        perc.append(np.percentile(simulator.capital,risk))
    glide_paths[t] = np.argmax(perc)

ValueError: operands could not be broadcast together with shapes (10001,2) (1001,2) 

In [ ]:
np.take(portfolios,list(glide_paths.values()),0)

array([[0.2, 0.8],
       [0.3, 0.7],
       [0.3, 0.7],
       [0.4, 0.6],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.8, 0.2],
       [0.8, 0.2],
       [0.8, 0.2],
       [0.9, 0.1],
       [0.9, 0.1],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ]])

In [ ]:


capital = np.full(10001,100)

T = np.arange(11,120,12)
risk = 10


strategy = np.take(portfolios,list(glide_paths.values()),0)
strategy = np.repeat(strategy,12,axis=0)
simulator = Simulation(capital,sim_returns,strategy)
simulator.run(216)
perc = np.percentile(simulator.capital,risk)
gt = 50000

perc = perc/capital[0]

gt/perc


24831.148192292407

In [ ]:
ret = sim_returns
mu = ret.mean(1)
cov = np.zeros((1001,2,2))
for i in range(ret.shape[0]):
    cov[i]= np.cov(ret[i],rowvar=False)

In [ ]:
np.cov(ret[0],rowvar=False)

array([[9.16444603e-04, 3.73971440e-07],
       [3.73971440e-07, 1.81429710e-04]])

In [ ]:
portfolio = rp.Portfolio(pd.DataFrame(pln_returns[assets]))
#pln_returns[assets]

portfolio.mu = expected_returns
portfolio.cov = cov_matrix

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'MinRisk' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = False # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = portfolio.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

In [ ]:
means = expected_returns[['ISAC.L','^TBSP']]
cov = acwi_tbsc.cov()
number_of_scenarios = 101
T = 60


sim_returns = []

for s in range (number_of_scenarios):
    sim_returns.append(multivariate_normal.rvs(means,cov, T))

sim_returns = np.array(sim_returns)

NameError: name 'expected_returns' is not defined

In [ ]:
start = '01.12.2006'
tbsp = loadStooqData('^TBSP',start,'m')
wibor3 = loadStooqData('PLOPLN3M',start,'m')
cpi = loadStooqData('cpimpl.m',start,'m')

In [ ]:
tbsp = np.log(tbsp/tbsp.shift(1))
wibor3 = wibor3/100/12
cpi = cpi/100

In [ ]:
tbsp.loc['2007-01-01':].mean()*12
#'2019-12-31'

0.03856047611547877

In [ ]:
wibor3.loc['2007-01-01':].mean() * 12

0.032938

In [ ]:
cpi.loc['2007-01-01':].mean() *12

0.036306122448979614

In [ ]:
print(tbsp.loc['2007-01-01':'2023-12-31'].mean()*12)
print(wibor3.loc['2007-01-01':'2019-12-31'].mean() * 12)
print(cpi.loc['2007-01-01':'2019-12-31'].mean() *12)

0.03856047611547877
0.03302307692307694
0.02115384615384617


In [ ]:
tbsp_cum = (1+tbsp.loc['2007-01-01':'2023-06-31']).cumprod()[-1]
cpi_cum = (1.00125+cpi.loc['2007-01-01':'2023-06-31']).cumprod()[-1]
wibor_cum = (1+wibor3.loc['2007-01-01':'2023-06-31']/12).cumprod()[-1]

In [ ]:
print(tbsp_cum)
print(cpi_cum)
print(wibor_cum)

1.8398585720934704
2.3014767048682354
1.045825968849171


In [ ]:
tbsp_cum/cpi_cum
#wibor_cum/cpi_cum

0.799425242150694

In [86]:
assets = ['ACWI', '^TBSP', 'XAUPLN', 'MWIG40TR', 'CORP.L', 'PLOPLN3M']
means = expected_returns[assets]
cov = pln_returns[assets].cov()
number_of_scenarios = 1001
T = 60
sim_returns = []

for s in range (number_of_scenarios):
    sim_returns.append(multivariate_normal.rvs(means,cov, T))

sim_returns = np.array(sim_returns)

In [87]:
model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'
points = 15
weights = np.zeros((number_of_scenarios,len(assets),points))

for p in range(number_of_scenarios):
    portfolio = rp.Portfolio(pd.DataFrame(sim_returns[p], columns=assets))
    portfolio.assets_stats(method_mu='hist', method_cov='hist')
    w = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)
    weights[p,:] = w.to_numpy()


In [89]:
w_means = np.round(weights.mean(0),2)
w_means_df = pd.DataFrame(w_means.T, columns=assets)

In [90]:
w_means_df

,ACWI,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M
0,0.00,0.00,0.00,0.00,0.01,0.98
1,0.05,0.07,0.03,0.03,0.02,0.79
2,0.08,0.12,0.05,0.06,0.04,0.65
3,0.12,0.17,0.08,0.08,0.05,0.51
4,0.15,0.18,0.10,0.11,0.06,0.39
5,0.18,0.18,0.13,0.14,0.08,0.30
6,0.22,0.16,0.15,0.16,0.08,0.23
7,0.25,0.14,0.18,0.19,0.07,0.17
8,0.27,0.11,0.21,0.21,0.07,0.12
9,0.28,0.09,0.24,0.24,0.06,0.09


In [ ]:
sim_returns.mean(1) * 12

array([[0.07970701, 0.03594519],
       [0.13821283, 0.04863202],
       [0.04961046, 0.02331223],
       [0.04724664, 0.01191947],
       [0.05141667, 0.04384099],
       [0.11690707, 0.05204425],
       [0.10728094, 0.03374413],
       [0.07218974, 0.03473295],
       [0.11561089, 0.01857777],
       [0.08200223, 0.0519025 ],
       [0.04469573, 0.04964911]])